# Scraping mine data with `.apply`

## The pages we'll be looking at

If I wanted to read specific information about a specfic mine, it takes a few steps. **Do these steps with your browser before you try any programming.**

1. Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm)
2. Scroll down to **Mine Identification Number (ID) Search**
3. Type in a mine ID number, such as `3503598`, click **Search**
4. I'm on a page! It lists the MINE NAME and MINE OWNER.

After searching for and finding a mine, I can use this page to **find reports about this mine**. Some of the reports are on accidents, violations, inspections, health samples and more. To get those reports:

1. Search for a mine (if you haven't already)
2. Scroll down and change **Beginning Date** to `1/1/1995` (violation reports begin in 1995, accidents begin in 1983)
3. Select the report type of `Violations`
4. Click **Get Report**
5. I'm on a page! It lists ALL OF THE MINE'S VIOLATIONS.

By changing the report type you're searching for you can find all sorts of different data.

# Doing this programmatically

## First, scraping a single page

### Import your imports

In [ ]:
import pandas as pd
import time
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

### Searching for a mine

Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm) and use Selenium to search for `3503598`

- *TIP: You might need to use the Selenium code to scroll down to the right spot on the page. Or not!*
- *TIP: Use `.send_keys` to type into the box*
- *TIP: On pages that never change, you can usually just use XPath if you're feeling lazy*

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://arlweb.msha.gov/drs/drshome.htm')
time.sleep(3)

In [4]:
searchfld = driver.find_element_by_name('MineId')
driver.execute_script("arguments[0].scrollIntoView(true)", searchfld)

In [5]:
searchfld.send_keys(3503598)
time.sleep(1)
searchfld.send_keys(Keys.RETURN)

### Finding reports

On the "Report Selection Page" (where you should be after you search), use Selenium to...

- Change the **Beginning Date** to `1/1/1995`
- Select the report type of `Violations`
- Click **Get Report**

.

- *TIP: Remember, if someone isn't on the page Selenium can't click it!*

In [6]:
date_input = driver.find_element_by_name('BDate')
date_input.send_keys('1/1/1995')
# select violations radio button by xpath
driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input').click()
date_input.send_keys(Keys.RETURN)

In [ ]:
info_raw = driver.find_elements_by_tag_name('tr')
# I could've gone for .find_elements_by_class_name('drsviols') as well.
# My approach is probably less organized but also much less typing.
violation_lines = [vio.text.split() for vio in info_raw[31:] if vio.text != None] 
violation_lines

### Saving reports

Save all of the rows of data on that page into a new dataframe. Each column is its own column, **and you also need to save the URL under the 'Standard' column.** Here, I even made you a blank dictionary:

```python
data = {}
data['violator'] = ''
data['contract_id'] = ''
data['citation_no'] = ''
data['case_no'] = ''
data['date_issues'] = ''
data['final_order_date'] = ''
data['section_of_act'] = ''
data['date_terminated'] = ''
data['citation'] = ''
data['s_and_s'] = ''
data['standard'] = ''
data['standard_url'] = ''
data['proposed_penalty'] = ''
data['citation_status'] = ''
data['current_penalty'] = ''
data['amount_paid'] = ''
```

- *TIP: Some of those table rows aren't what you want. How can you tell them apart from the good ones? (the previous mine owner ones are okay, I just mean the weird headers)*
- *TIP: I sense `.find_elements` + a lot of square brackets*
- *TIP: This is just like scraping a search results page!*
- *TIP: For the URL, you'll need to find the `a` inside of the cell*
- *TIP: class name is sadly not going to save your life here, because some of the `tr`s and `td`s have the same class! It's stupid. But there's a trick: CSS selectors! Something like `div#container` finds a `div` with the id of `container`, while `span.important` finds a `span` with the class of `important`. It should be helpful! And use `.find_elements_by_` + tab to see what the command is*

In [ ]:
info_raw = driver.find_elements_by_tag_name('tr')
# The split is making things complicated for now, but I can grab every item whenever I want
# The [31:] cuts of the header and stuff above the data
violation_lines = [vio.text.split() for vio in info_raw[31:] if vio.text != ''] 
all_datas =[]
for violine in violation_lines:

    data = {}
    try:
        data['violator'] = ' '.join(violine[0:-13])
        # Will take care of this later, for now all are empty
        data['contract_id'] = np.nan
        data['citation_no'] = violine[-13]
        data['case_no'] = violine[-12]
        data['date_issues'] = violine[-11]
        data['final_order_date'] = violine[-10]
        data['section_of_act'] = violine[-9]
        data['date_terminated'] = violine[-8]
        data['citation'] = violine[-7]
        data['s_and_s'] = violine[-6]
        data['standard'] = violine[-5]
        # The url thingy is very lazy, I know...
        data['standard_url'] = driver.find_element_by_partial_link_text(violine[-5]).get_attribute('href') 
        data['proposed_penalty'] = violine[-4]
        data['citation_status'] = violine[-3]
        data['current_penalty'] = violine[-2]
        data['amount_paid'] = violine[-1]
    except:
        print('ERROR:',violine)
    #print(data)
    all_datas.append(data)

all_datas

### Saving that data

Save the dataframe to a CSV file called `3503598-violations.csv` (that's the TDLR code)

In [ ]:
df = pd.DataFrame(all_datas)
# There is one empty cell left over from the table headers:
df.dropna(thresh = 10, inplace=True) 
df.head(5)
df.to_csv('3503598-violations.csv', index = False)

## Put that all in ONE cell that runs correctly

The **entire process**, from searching to saving as a CSV

In [ ]:
driver = webdriver.Chrome()
driver.get('https://arlweb.msha.gov/drs/drshome.htm')
time.sleep(3)

searchfld = driver.find_element_by_name('MineId')
driver.execute_script("arguments[0].scrollIntoView(true)", searchfld)
searchfld.send_keys(3503598)
time.sleep(1)
searchfld.send_keys(Keys.RETURN)

date_input = driver.find_element_by_name('BDate')
date_input.send_keys('1/1/1995')
driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input').click()
date_input.send_keys(Keys.RETURN)

info_raw = driver.find_elements_by_tag_name('tr')
violation_lines = [vio.text.split() for vio in info_raw[28:] if vio.text != ''] 
all_datas =[]
errorcount = 0
for violine in violation_lines:

    data = {}
    try:
        data['violator'] = ' '.join(violine[0:-13])
        data['contract_id'] = np.nan
        data['citation_no'] = violine[-13]
        data['case_no'] = violine[-12]
        data['date_issues'] = violine[-11]
        data['final_order_date'] = violine[-10]
        data['section_of_act'] = violine[-9]
        data['date_terminated'] = violine[-8]
        data['citation'] = violine[-7]
        data['s_and_s'] = violine[-6]
        data['standard'] = violine[-5]
        # The url thingy is very lazy, I know...
        data['standard_url'] = driver.find_element_by_partial_link_text(violine[-5]).get_attribute('href') 
        data['proposed_penalty'] = violine[-4]
        data['citation_status'] = violine[-3]
        data['current_penalty'] = violine[-2]
        data['amount_paid'] = violine[-1]
    except:
        # This is for debugging only. With most mine data sets, there are 4 errors from the header rows.
        errorcount = errorcount + 1
    all_datas.append(data)


print(errorcount)
df = pd.DataFrame(all_datas)
# There is a bit of noise. I drop it here:
df.dropna(thresh = 12, inplace=True) 
driver.close()
df.head(5)


# Using .apply to find data about SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [ ]:
df = pd.read_csv('mines-subset.csv')
df.head()

### Open up `mines-subset.csv` in a text editor, then look at your dataframe. Is something different about them? If so, make them match.

- *TIP: You can zero fill if you want, but another option is that when reading in a CSV, `dtype='str'` will force everything to be a string*

In [ ]:
df = pd.read_csv('mines-subset.csv', dtype = 'str')
df.head()

### Convert your one-cell scraper into a function, and use it on each row of our dataset

- *TIP: You'll be using `.apply`*
- *TIP: You won't be joining this back into your dataframe, so you don't need to `return` anything or `join` or any of that.*
- *TIP: Be careful of your **other variable names** - if you're calling the thing you're sending your function `row`, you can't use it anywhere else (like in your loop)*
- *TIP: **BE CAREFUL WHAT YOU NAME YOUR DATAFRAMES.** If you name the citations dataframe `df` it can overwrite your mine ID `df`*
- *TIP: You'll be saving a dataframe each time*
- *TIP: Be sure you change everything that refers to the mine ID to refer to the current row's ID instead of `3503598`*
- *TIP: BE SURE TO CHANGE EVERYTHING THAT REFERS TO THE MINE ID*
- *TIP: EVERYTHING, EVERYTHING, EVERYTHING! Look at the end of your function! Maybe I'm overreacting, I don't know.*
- *TIP: If you hit an error about list index out of range, see what line it's happening on and go look at the page. What's different about this page than the previous ones? (answer: the last three columns!) If you assign those columns later using `try`/`except` you should be able to get some data from those rows without throwing it all out. If you can't figure it out, just wrap it all in try/except and give up on those rows*
- *TIP: Some of the standards might not have links, either, so you might want to wrap that in a `try`/`except`, too!*

# Okay, now do it for ALL of the mines

Open up `mines.csv` using pandas and do the same thing, it will just be for more mines this time.